# Virtual Piano Tutor

## Project Description

This is a notebook for tracking my progress on VPT...

- Best Classifier as of 11/30
    - SVM {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

## TODO List

- TODAY
    - Decide on RDF model to keep for rest of project
    - Work on RDF data and annotations
    - Add results to file
    - Rewrite RDF for GridSearchCV
        - Extend RDF
    - Work on ideas for paper
        - Visualizations
    - Play with CAE
    - How to automate this...
    - Windowing/Summarizing
    
- DONE
    - ~~Organize RDF data~~
    - ~~Generate data from already extracted hands...~~
    - ~~Get notebook running on Compute Canada~~
    - ~~Get data on Compute Canada~~
    - ~~Setup CAE to deal with hand images~~
    - ~~setup data for training autoencoder on LH and RH~~
    - ~~Train Autoencoder for LH and Rh~~
    

- Bad Segmentation
    - p3c - left hand (not terrible)
    - p1s - right hand (shouldn't use)
    - p5a - Both could use some work but still caputures most of the left hand (RH not so good...)
    - p5c - not good (left hand passable...)
    
- Add noise to CAE
    - http://scikit-image.org/docs/dev/api/skimage.util.html#random-noise
    
- ~~Multiple Participants~~
    - ~~have one holdout set participant~~
        - ~~Test with p1&2 training p3 testing, then p1&3...~~
    - ~~have one holdout set exercise~~

- Test with RH too

- Windowing data
    - Summarize data for classification
    - Majority Voting (or with probabilities)

- Look for other features
    - Others??
    - ~~Autoencoder features~~
    - ~~HONV~~
    
- Work on hand segmentation
   - See p1e for bad examples
   - How to validate segmentation?
       - Statistical analysis on length and width ratios
       
- Visualize !!!
    - Input 
    - Results !!!
        - F Scores
        - Accuracy
        - Try weighted instead of macro




- Finish Project Description

- ~~Turn into functions~~
    
- ~~Verify Segmentation~~
    - have only done basic verification
    
- ~~FIRST THING: Test by ignoring training data (p1s) and then using train_test_split on recordings~~
    - ~~Data should be ready for spliting~~
    
- ~~Remove data from testing to find culprit~~
    
- ~~Track my progress better !!! (duh through notebooks!)~~

# Setup

## Libraries

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s
import vpt.hand_detection.depth_context_features as dcf

%load_ext autoreload
%autoreload 2

## Some helper functions

### Generate Dataset

In [ ]:
def generate_data(hs, feature_type="hog"):

    X_lh = []
    y_lh = []

    X_rh = []
    y_rh = []

    filenames = []
    
    percent = .01

    hgen = hg.hand_generator(debug=True)
    hg_size = hg.size()
    for i, (lh, rh) in enumerate(hgen):
        
        if lh.label() != None and rh.label() != None:

            filenames.append(lh.get_fpath())
            y_lh.append(lh.label())
            x_lh = extract_features(lh.get_hand_img(), feature_type, visualise=False)
            X_lh.append(x_lh)
            
            y_rh.append(rh.label())
            x_rh = extract_features(rh.get_hand_img(), feature_type, visualise=False)
            X_rh.append(x_rh)
            
        else:
            raise RuntimeWarning("Warning: No label found for hands")
        
        if i / hg_size > percent:
            print("#", end="")
            percent += .01
                

    X_lh = np.array(X_lh)
    y_lh = np.array(y_lh)
    X_rh = np.array(X_rh)
    y_rh = np.array(y_rh)
    filenames = np.array(filenames)
    
    return X_lh, y_lh, X_rh, y_rh, filenames

### Load/Save Data Set

In [ ]:
def save_data(X_lh, y_lh, X_rh, y_rh, filenames, testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    
    data_path = os.path.join(base, "{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    np.savez_compressed(data_path + data_type + "_data.npz" , X_lh=X_lh, y_lh=y_lh, X_rh=X_rh, y_rh=y_rh, filenames=filenames)

In [35]:
def load_data(testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    data_path = os.path.join(base, "{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    data = np.load(data_path + data_type + "_data.npz")    
    return data

## Project Setup

In [36]:
## Some General Parameters
s.participant = "all"
s.sensor = "realsense"

participants = ["p1", "p3", "p4", "p6"]
posture_folders = {p : os.path.join("data/posture", p) for p in participants}

ftype = ".bin"
folders = "data/posture/"

In [37]:
## Model Parameters
refreshHD = False
refreshCLF = True

## RDF Parameters
M = 5
radius = .15

## Posture Detection Parameters
feature_type = "shog2"
annotation_file = "data/posture/annotations.txt"

offset_gen = dcf.generate_feature_offsets
feature_gen = dcf.calc_features

### Generate or Load Data

In [ ]:
#### Generate and Save data for all testing participants
folders = [folder for p, folder in posture_folders.items()]
print(folders)

hg = init_hand_generator(folders, "all", annotation_file, offset_gen, feature_gen, 
                   M, radius, n_samples=750, ftype=".bin")

print("\n### Generating Data Set ###")
X_lh, y_lh, X_rh, y_rh, filenames = generate_data(hg, feature_type=feature_type)
save_data(X_lh, y_lh, X_rh, y_rh, filenames, "all_participants-blarg", M, radius, feature_type, data_type="train")

In [38]:
#### Load data for a single paricipant
data = load_data("all_participants", M, radius, feature_type, "train")
print("X LH", data["X_lh"].shape, "y LH", data["y_lh"].shape, data["vis_lhs"].shape)
print("X RH", data["X_rh"].shape, "y RH", data["y_rh"].shape, data["vis_rhs"].shape)
print("Filenames", data["filenames"].shape)

X LH (15818, 342) y LH (15818,) (15818, 120, 90)
X RH (15818, 342) y RH (15818,) (15818, 120, 90)
Filenames (15818,)


In [7]:
## using p6 for validation
r = re.compile('p3')
vmatch = np.vectorize(lambda x:bool(r.search(x)))
val_p = vmatch(data['filenames'])

X_lh = data['X_lh'][~val_p]
y_lh = data['y_lh'][~val_p]
X_rh = data['X_rh'][~val_p]
y_rh = data['y_rh'][~val_p]
filenames = data['filenames'][~val_p]


X_lh_test = data['X_lh'][val_p]
y_lh_test = data['y_lh'][val_p]
X_rh_test = data['X_rh'][val_p]
y_rh_test = data['y_rh'][val_p]
filenames_test = data['filenames'][val_p]

print("Cross Val Data:")
print(X_lh.shape, y_lh.shape)
print(X_rh.shape, y_rh.shape)
print(filenames.shape)
print()
print("Validation Data")
print(X_lh_test.shape, y_lh_test.shape)
print(X_rh_test.shape, y_rh_test.shape)
print(filenames_test.shape)

Cross Val Data:
(11246, 180) (11246,)
(11246, 180) (11246,)
(11246,)

Validation Data
(4572, 180) (4572,)
(4572, 180) (4572,)
(4572,)


In [8]:
groups = np.zeros_like(filenames, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups[np.where(np.char.find(filenames, p) != -1)] = p_num
    
print(groups.shape)
print(np.unique(groups))
print(groups)

(11246,)
[1 4 6]
[1 1 1 ..., 6 6 6]


# Classification

### Libraries

In [18]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, LeaveOneGroupOut
from imblearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

## Load Data for Classification

In [10]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)
rem_static_test = vmatch(filenames_test)

X_lh_train, y_lh_train, filenames_train, groups_train = X_lh[~rem_static], y_lh[~rem_static], filenames[~rem_static], groups[~rem_static]
X_rh_train, y_rh_train, filenames_train, groups_train = X_rh[~rem_static], y_rh[~rem_static], filenames[~rem_static], groups[~rem_static]

In [11]:
X_lh_val, y_lh_val, filenames_val = X_lh_test[~rem_static_test], y_lh_test[~rem_static_test], filenames_test[~rem_static_test]
X_rh_val, y_rh_val, filenames_val = X_rh_test[~rem_static_test], y_rh_test[~rem_static_test], filenames_test[~rem_static_test]

In [12]:
X_comb_train = np.vstack((X_lh_train, X_rh_train))
y_comb_train = np.hstack((y_lh_train, y_rh_train))
groups_comb_train = np.hstack((groups_train, groups_train))
filenames_comb_train = np.hstack((filenames_train, filenames_train))

X_comb_val = np.vstack((X_lh_val, X_rh_val))
y_comb_val = np.hstack((y_lh_val, y_rh_val))
filenames_comb_val = np.hstack((filenames_val, filenames_val))

In [13]:
print("Cross Validation Data")
print("LH:", X_lh_train.shape, y_lh_train.shape)
print(np.unique(y_lh_train, return_counts=True))
print("RH:", X_rh_train.shape, y_rh_train.shape)
print(filenames_train.shape)
print(groups_train.shape)
print(np.unique(y_rh_train, return_counts=True))
print("Combined:", X_comb_train.shape, y_comb_train.shape)
print(filenames_comb_train.shape)
print(groups_comb_train.shape)
print(np.unique(y_comb_train, return_counts=True))
print()
print()
print("Validation Data")
print("LH:", X_lh_val.shape, y_lh_val.shape)
print(np.unique(y_lh_val, return_counts=True))
print("RH:", X_rh_val.shape, y_rh_val.shape)
print(np.unique(y_rh_val, return_counts=True))
print("Combined:", X_comb_val.shape, y_comb_val.shape)
print(np.unique(y_comb_val, return_counts=True))
print(filenames_comb_val.shape)

Cross Validation Data
LH: (9629, 180) (9629,)
(array([0, 1, 2]), array([6890, 1718, 1021]))
RH: (9629, 180) (9629,)
(9629,)
(9629,)
(array([0, 1, 2]), array([7496, 2106,   27]))
Combined: (19258, 180) (19258,)
(19258,)
(19258,)
(array([0, 1, 2]), array([14386,  3824,  1048]))


Validation Data
LH: (2650, 180) (2650,)
(array([0, 1, 2]), array([1267, 1336,   47]))
RH: (2650, 180) (2650,)
(array([0]), array([2650]))
Combined: (5300, 180) (5300,)
(array([0, 1, 2]), array([3917, 1336,   47]))
(5300,)


## Model Testing

### SVM

In [ ]:
## Parameters for SVMs
steps = [('SMOTE', SMOTE()), ("SVC", SVC())]
param_grid = [
  {'SVC__C': [1, 10, 100, 1000], 'SVC__kernel': ['linear'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
  {'SVC__C': [1, 10, 100, 1000], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
 ]

# steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
# param_grid = [
#   {'SVC__C': [1, 10, 100, 1000], 'SVC__kernel': ['linear'], 'SMOTE__smote': [SMOTE(kind='regular'), SMOTE(kind='borderline1'), SMOTE(kind='borderline2'), SMOTE(kind='svm')]},
#   {'SVC__C': [1, 10, 100, 1000], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__smote': [SMOTE(kind='regular'), SMOTE(kind='borderline1'), SMOTE(kind='borderline2'), SMOTE(kind='svm')]},
#  ]


pipeline = Pipeline(steps)

scores = ['accuracy']
# logo = LeaveOneGroupOut()

In [ ]:
# Hyper Parameter Tuning
for score in scores:
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print("## Tuning hyper-parameters for {}".format(score))
        print()

        if score is "accuracy":
            scoring = score
        else:
            scoring = '{}_macro'.format(score)
    
        #### TRAIN LH
        clf_lh = GridSearchCV(pipeline, param_grid, cv=3, scoring=scoring, n_jobs=2, verbose=2)
        clf_lh.fit(X_lh_train, y_lh_train)

        print("Best LH Parameters set found on data set:")
        print()
        print(clf_lh.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_lh.cv_results_['mean_test_score']
        stds  = clf_lh.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_lh.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()
        print()
        
        
#         #### TRAIN RH
#         clf_rh = GridSearchCV(pipeline, param_grid, cv=3, scoring=scoring, n_jobs=2)
#         clf_rh.fit(X_rh_train, y_rh_train)

#         print("Best RH Parameters set found on data set:")
#         print()
#         print(clf_rh.best_params_)
#         print()
#         print("Grid scores on data set:")
#         print()
#         means = clf_rh.cv_results_['mean_test_score']
#         stds  = clf_rh.cv_results_['std_test_score']
#         for mean, std, params in zip(means, stds, clf_rh.cv_results_['params']):
#             print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
#         print()
        
        
#         #### TRAIN COMBINED LH & RH
#         clf_comb = GridSearchCV(pipeline, param_grid, cv=3, scoring=scoring, n_jobs=2)
#         clf_comb.fit(X_comb_train, y_comb_train)

#         print("Best Combined Parameters set found on data set:")
#         print()
#         print(clf_comb.best_params_)
#         print()
#         print("Grid scores on data set:")
#         print()
#         means = clf_comb.cv_results_['mean_test_score']
#         stds  = clf_comb.cv_results_['std_test_score']
#         for mean, std, params in zip(means, stds, clf_comb.cv_results_['params']):
#             print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
#         print()

In [ ]:
from sklearn.metrics import accuracy_score
y_lh_true, y_lh_pred = y_lh_val, clf_lh.predict(X_lh_val)
# y_rh_true, y_rh_pred = y_rh_val, clf_rh.predict(X_rh_val)
# y_comb_true, y_comb_pred = y_comb_val, clf_comb.predict(X_comb_val)
print("LH Validatation Score:", accuracy_score(y_lh_true, y_lh_pred))
print("LH Confustion Matrix:\n", confusion_matrix(y_lh_true, y_lh_pred))
print()
# print("RH Validatation Score:", accuracy_score(y_rh_true, y_rh_pred))
# print("RH Confustion Matrix:\n", confusion_matrix(y_rh_true, y_rh_pred))
# print()
# print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
# print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))


### Single Classifier

In [32]:
steps_proba = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.0001, kernel='linear', probability=False))]
pipeline_proba = Pipeline(steps_proba)
pipeline_proba.fit(X_rh_train, y_rh_train)

Pipeline(memory=None,
     steps=[('SMOTE', SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None)), ('SVC', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [33]:
y_rh_pred = pipeline_proba.predict(X_rh_val)

In [34]:
y_rh_true, y_rh_pred = y_rh_val, y_rh_pred
print("rH Validatation Score:", accuracy_score(y_rh_true, y_rh_pred))
print("rH Confustion Matrix:\n", confusion_matrix(y_rh_true, y_rh_pred))

rH Validatation Score: 0.51358490566
rH Confustion Matrix:
 [[1361 1277   12]
 [   0    0    0]
 [   0    0    0]]


In [29]:
y_rh_true_maj = []
for i in range(0, y_rh_true.size, 10):
    u, counts = np.unique(y_rh_true[i:i+10], return_counts=True)
    pred = u[np.argmax(counts)]
    y_rh_true_maj.append(pred)
    
y_rh_true_maj = np.array(y_rh_true_maj)

In [30]:
y_rh_pred_maj = []
for i in range(0, y_rh_pred.size, 10):
    u, counts = np.unique(y_rh_pred[i:i+10], return_counts=True)
    pred = u[np.argmax(counts)]
    y_rh_pred_maj.append(pred)
    
y_rh_pred_maj = np.array(y_rh_pred_maj)

In [31]:
print("rH Validatation Score:", accuracy_score(y_rh_true_maj, y_rh_pred_maj))
print("rH Confustion Matrix:\n", confusion_matrix(y_rh_true_maj, y_rh_pred_maj))

rH Validatation Score: 0.0
rH Confustion Matrix:
 [[  0 253  12]
 [  0   0   0]
 [  0   0   0]]


In [39]:
radii = np.linspace(50000, 1000000, 9)

In [40]:
radii

array([   50000.,   168750.,   287500.,   406250.,   525000.,   643750.,
         762500.,   881250.,  1000000.])